# Imports

In [ ]:
import numpy as np
import pandas as pd
from scipy import stats
#importing the models
import Kmeans
import ALS
import NN

# Final Functions

In [ ]:
def median_vote(predictions):
    #computing the median
    median=np.median(predictions,axis=0)
    #making sure the result is an int (not the case if the amount of predictions is even)
    bounded_median=np.floor(median)
    return bounded_median

In [ ]:
def mode_vote(predictions):
    #computing the mode
    return stats.mode(predictions,axis=0)[0]

In [69]:
def vote(voting_f):
    #useful constants
    submission_path='submission.csv'
    training_path = "data/data_train.csv"
    format_path = "data/sampleSubmission.csv"
    #computing the prediction of the ALS algorithm
    predictions=ALS.main(training_path, format_path)
    #computing multiple predictions of the kmeans algorithm
    for k in [3,6,7]:
        predictions=predictions.merge(Kmeans.main(training_path, format_path, k),on='Id')
    #computing the prediction of the NN algorithm
    predictions=predictions.merge(NN.main(training_path, format_path),on='Id')
    #setting 'Id' as the index of the aggregation of predictions
    predictions.set_index('Id', inplace=True)
    #finding the best prediction through the voting function
    print('Voting...')
    predictions['Prediction']=median_vote(predictions.T)
    #exporting the final prediction using the submission path
    print('Exporting the final prediction...')
    predictions[['Prediction']].to_csv(submission_path)
    print('Done!')

In [70]:
vote(median_vote)

Loading datasets
Spliting train/test
the shape of original ratings. (# of row, # of col): (10000, 1000)
Splitting progression: 100.0%
Total number of nonzero elements in original data:1,176,952
Total number of nonzero elements in train data:1,065,327
Total number of nonzero elements in test data:111,625
Trying to retrieve cached optimal matrix factorization
Successfully retrieved cached optimal matrix factorization
Emitting predictions 1176952/1176952
Kmeans for k= 7 :
The current iteration of k-means is: 49, the average loss is 113.39338894580459.


C:\Users\ajlre\Anaconda3\lib\site-packages\tensorflow_core\python\framework\indexed_slices.py:424: UserWarning: Converting sparse IndexedSlices to a dense Tensor of unknown shape. This may consume a large amount of memory.
  "Converting sparse IndexedSlices to a dense Tensor of unknown shape. "


Train on 1059256 samples, validate on 117696 samples
Epoch 1/10
1059256/1059256 [==============================] - 20s 19us/step - loss: 1.0341 - val_loss: 1.0090
Epoch 2/10
1059256/1059256 [==============================] - 24s 23us/step - loss: 1.0067 - val_loss: 1.0029
Epoch 3/10
1059256/1059256 [==============================] - 24s 22us/step - loss: 0.9995 - val_loss: 1.0005
Epoch 4/10
1059256/1059256 [==============================] - 24s 22us/step - loss: 0.9948 - val_loss: 1.0022
Epoch 5/10
1059256/1059256 [==============================] - 24s 22us/step - loss: 0.9915 - val_loss: 0.9989
Epoch 6/10
1059256/1059256 [==============================] - 23s 22us/step - loss: 0.9888 - val_loss: 0.9977
Epoch 7/10
1059256/1059256 [==============================] - 24s 23us/step - loss: 0.9868 - val_loss: 1.0000
Epoch 8/10
1059256/1059256 [==============================] - 24s 22us/step - loss: 0.9847 - val_loss: 0.9992
Epoch 9/10
1059256/1059256 [==============================] - 24s 2

# Experimental Code

In [ ]:
#useful constants
submission_path='submission.csv'
training_path = "data/data_train.csv"
format_path = "data/sampleSubmission.csv"

In [ ]:
#computing the predictions of the ALS algorithm
predictions_als=ALS.main(training_path, format_path).set_index("Id")

In [ ]:
#computing the best prediction of the kmeans algorithm
k=7
predictions_kmeans = Kmeans.main(training_path, format_path, k).set_index("Id")

In [ ]:
#computing the prediction of the NN algorithm
predictions_nn = NN.main(training_path, format_path).set_index("Id")

In [ ]:
#finding the best prediction through the voting function
concat = pd.merge(pd.merge(predictions_kmeans, predictions_als, left_index=True, right_index=True), predictions_nn, left_index=True, right_index=True)
concat.columns = ["Pred1", "Pred2", "Pred3"]
concat["Prediction"] = concat.median(axis=1) #voting function
concat.drop(["Pred1", "Pred2", "Pred3"], axis=1, inplace=True)

In [ ]:
#exporting the final prediction using the submission path
concat.to_csv(submission_path)